In [1]:
import pandas as pd
import time
from multiprocessing import Pool

In [2]:
def preProcessData(stockName):
    df = pd.read_csv("IntradayData_2018/"+stockName+".txt")
    df.columns = ['stock', 'date','time','openingPrice','high','low','closingPrice','volume','e']
    df = df.drop('e', 1)
    ST = [1,2,4,8,12,16,20,30,40]
    LT = [80,90,100,150,200,300,400,500, 750, 1000]
    for i in ST:
        df['MA_'+str(i)] = df['openingPrice'].rolling(window=i).mean()
    for i in LT:
        df['MA_'+str(i)] = df['openingPrice'].rolling(window=i).mean()
#     df['MA_ST'] = df['openingPrice'].rolling(window=MA_ST).mean()
#     df['MA_LT'] = df['openingPrice'].rolling(window=MA_LT).mean()
    dateList = list(set(df.date.values))
    dateList = sorted(dateList, key=abs, reverse=False)
#     print(df[:1])
    return df, dateList

In [3]:
def convertTime(timeString):
    timeArr = timeString.split(':')
    time = int(timeArr[0])*60 + int(timeArr[1])
    return time

def revConvertTime(timeInt):
    timeHr = timeInt/60
    timeMin = timeInt%60
    return str(timeHr) + ':' + str(timeMin)

def sellIt(buyPrice, sellPrice, timeBought, timeSold, verbose):
    profit = 100*(sellPrice - buyPrice*(1.0002))/(1.0*buyPrice )
    if(verbose): 
        print(buyPrice, sellPrice, revConvertTime(timeBought), revConvertTime(timeSold), profit)
    return profit

def buyIt(sellPrice, buyPrice, timeBought, timeSold, verbose):
    profit = 100*(sellPrice - buyPrice*(1.0002))/(1.0*buyPrice )
    if(verbose): 
        print(buyPrice, sellPrice, revConvertTime(timeBought), revConvertTime(timeSold), profit)
    return profit

def parseRow(row, ST, LT):
    time = convertTime(row[1].time)
    openingPrice = row[1].openingPrice
    low = row[1].low
    high = row[1].high
    closingPrice = row[1].closingPrice
    listPrice = 0.5*(openingPrice + closingPrice)
    diff = row[1]["MA_"+str(LT)] - row[1]["MA_"+str(ST)]
    return time, openingPrice, low, high, closingPrice, listPrice, diff

In [4]:
def dayScript(df, ST, LT, money, targetPercentage, stopLossPercentage, entryDifference, verbose = False):
    eomDays = 0
    boughtFlag = 0
    dailyTrades = 0
    buyPrice = 0
    sellPrice = 0
    sellMargin = 1 + targetPercentage
    buyMargin = 1 - targetPercentage

    stopLoss = 1 - stopLossPercentage
    stopGain = 1 + stopLossPercentage

    for row in df.iterrows():
        time, openingPrice, low, high, closingPrice, listPrice, diff = parseRow(row, ST, LT)       
        if(time >= 570 and time <900):
            if(boughtFlag == 0):
                if(diff>0 and diff<entryDifference*listPrice):
                    buyPrice = listPrice
                    sellPrice = 0
                    timeBought = time
                    timeSold = 0
                    boughtFlag = 1
                elif(diff<0 and diff>-1*entryDifference*listPrice):
                    sellPrice = listPrice
                    buyPrice = 0
                    timeSold = time
                    timeBought = 0
                    boughtFlag = -1
            elif(boughtFlag==1):
                if(high > buyPrice*sellMargin):
                    percentageProfit = sellIt(buyPrice, buyPrice*sellMargin, timeBought, time, verbose)
                    money = money*(1 + percentageProfit/100)
                    dailyTrades+=1
                    boughtFlag = 0
                    buyPrice = 0
                elif(buyPrice*stopLoss > low):
                    percentageProfit = sellIt(buyPrice, buyPrice*stopLoss, timeBought, time, verbose)
                    money = money*(1 + percentageProfit/100)
                    dailyTrades+=1
                    boughtFlag = 0
                    buyPrice = 0
            else:
                if(sellPrice*buyMargin > low):
                    percentageProfit = buyIt(sellPrice, sellPrice*buyMargin, timeSold, time, verbose)
                    money = money*(1 + percentageProfit/100)
                    dailyTrades+=1
                    boughtFlag = 0
                    sellPrice = 0
                elif(high > sellPrice*stopGain):
                    percentageProfit = buyIt(sellPrice, sellPrice*stopGain, timeSold, time, verbose)
                    money = money*(1 + percentageProfit/100)
                    dailyTrades+=1
                    boughtFlag = 0
                    sellPrice = 0
        elif(time >= 900):
            if(boughtFlag == 1):
                percentageProfit =sellIt(buyPrice, closingPrice, timeBought, time, verbose)
                money = money*(1 + percentageProfit/100)
                dailyTrades+=1
                boughtFlag = 0
                buyPrice = 0
                eomDays += 1
            elif(boughtFlag == -1):
                percentageProfit = buyIt(sellPrice, closingPrice, timeSold, time, verbose)
                money = money*(1 + percentageProfit/100)
                dailyTrades+=1
                boughtFlag = 0
                buyPrice = 0
                eomDays += 1

#     print('Total Profit : ',totalProfit)
    return money, dailyTrades, eomDays

In [5]:
def movingAverageExperiment(paramList):
    df, dateList, ST,LT,targetPercentage,stopLossPercentage,entryDifference = paramList
    money = 1
    for date in dateList:
        new_df = df[(df.date == date)]
        updatedMoney, dailyTrades, eomDays = dayScript(new_df, ST, LT,  money, targetPercentage, stopLossPercentage, entryDifference, verbose = False)
#         print(date, money, updatedMoney, dailyTrades)
        money = updatedMoney

#     print('Eom days - ',eomDays)
    yearlyProfitPercentage = (money - 1) * 100
    print('Total Profit %= ', yearlyProfitPercentage)
   
    return yearlyProfitPercentage

In [6]:
# #old
# ST = [1,4,12,20]
# LT = [80,100,200,500]
# targetPercentage = [0.0005, 0.0010, 0.0015, 0.0020]
# stopLossPercentage = [0.002, 0.006, 0.008, 0.010]
# entryDifference = [0.00001, 0.00005, 0.00010, 0.0002]
# #new
# ST = [1,4,12,20]
# LT = [100,200,500,750,1000]
# targetPercentage = [0.0015, 0.0020, 0.003, 0.005]
# stopLossPercentage = [0.002, 0.006, 0.008, 0.010]
# entryDifference = [0.000001, 0.000005, 0.00001, 0.00005]
#v3
ST = [1,4,12,20]
LT = [200,500,1000,1500]
targetPercentage = [0.003,0.005,0.0075,0.01]
stopLossPercentage = [0.006, 0.007, 0.008, 0.009, 0.010]
entryDifference = [0.00001, 0.000025, 0.00005]

paramList = [LT, ST, targetPercentage, stopLossPercentage, entryDifference]

In [ ]:
# from sklearn.grid_search import ParameterGrid
from sklearn.model_selection import ParameterGrid
param_grid = {'ST' : ST, 'LT': LT, 'targetPercentage': targetPercentage, 'stopLossPercentage': stopLossPercentage, 'entryDifference':entryDifference,}

grid = ParameterGrid(param_grid)

# ST_list = []
# LT_list = []
# targetPercentageList = []
# stopLossPercentageList = []
# entryDifferenceList = []
csvList = []
df, dateList = preProcessData('RELIANCE')

threadPoolSize = 32
pool = Pool(threadPoolSize)
resultList = {}
i=0
print('param combination = ' ,len(grid))
while i<len(grid):
    param_list =[]
    for j in range(threadPoolSize):
        params = grid[i+j]
        param_list.append([df, dateList, params['ST'], params['LT'],params['targetPercentage'], params['stopLossPercentage'],params['entryDifference']])
#         result = pool.apply_async(movingAverageExperiment, [df, dateList, params['ST'], params['LT'],params['targetPercentage'], params['stopLossPercentage'],params['entryDifference']])
#         params['result'] = result.get(timeout)
#         csvList.append(params)
    start = time.time()
    resultList = pool.map(movingAverageExperiment, param_list)
    end = time.time()
    for j in range(threadPoolSize):
        params = param_list[j]
        result = resultList[j]
        newListVariable = {}
        newListVariable['params'] = params
        newListVariable['result']= result
        csvList.append(newListVariable)
    print('Time - ',end - start)
    i+=threadPoolSize
    print(i,len(grid))

('param combination = ', 960)
('Total Profit %= ', -4.853613533516842)
('Total Profit %= ', -1.6308562874737698)
('Total Profit %= ', -0.8615895628114156)
('Total Profit %= ', 0.4538251233864976)
('Total Profit %= ', -5.965635463181185)
('Total Profit %= ', -2.8783701198394795)
('Total Profit %= ', -4.853598560806782)
('Total Profit %= ', -7.825350135203624)
('Total Profit %= ', -2.0215422509619607)
('Total Profit %= ', -2.837214897142959)
('Total Profit %= ', -3.9188587877734604)
('Total Profit %= ', -5.596142523525005)
('Total Profit %= ', -2.1634311368312398)
('Total Profit %= ', -1.9968959098224714)
('Total Profit %= ', -1.2369756999561554)
('Total Profit %= ', -0.8201265613176356)
('Total Profit %= ', -5.556412518635279)
('Total Profit %= ', -1.4455634028514908)
('Total Profit %= ', -2.1069352700884725)
('Total Profit %= ', -1.9254780496187895)
('Total Profit %= ', -13.850471508629347)
('Total Profit %= ', -14.515721309612417)
('Total Profit %= ', -16.92614314485904)
('Total Profi

('Total Profit %= ', -4.4394310862238)
('Total Profit %= ', -2.581073276609802)
('Total Profit %= ', -1.170938330560467)
('Total Profit %= ', -2.738925197209685)
('Total Profit %= ', -5.003614144515156)
('Total Profit %= ', -3.247701442397033)
('Total Profit %= ', -1.847215911550748)
('Total Profit %= ', -3.4044731977781018)
('Total Profit %= ', -8.273721548772329)
('Total Profit %= ', 0.5608699148404339)
('Total Profit %= ', -3.6675011136406144)
('Total Profit %= ', -8.332816931325226)
('Total Profit %= ', -3.464798521528545)
('Total Profit %= ', -4.527297960368704)
('Total Profit %= ', -0.5625342938500699)
('Total Profit %= ', -11.048251738685977)
('Total Profit %= ', -4.716593074179687)
('Total Profit %= ', -7.131746136457961)
('Total Profit %= ', -3.2751392653299516)
('Total Profit %= ', -13.790480924880999)
('Total Profit %= ', -8.00627166922354)
('Total Profit %= ', -6.353110617011481)
('Total Profit %= ', -10.086992830033214)
('Total Profit %= ', -11.023491968896105)
('Total Pro

('Total Profit %= ', 4.105773561759074)
('Total Profit %= ', 2.9429669593715513)
('Total Profit %= ', -0.5881065952905495)
('Total Profit %= ', -1.1054034393146894)
('Total Profit %= ', 2.1482348980983)
('Total Profit %= ', 1.0072929704993516)
('Total Profit %= ', 0.028545706937266324)
('Total Profit %= ', -1.855568015261333)
('Total Profit %= ', 1.1584320586794261)
('Total Profit %= ', -2.866756787623559)
('Total Profit %= ', -1.8990057561928952)
('Total Profit %= ', -1.1517903011311725)
('Total Profit %= ', 1.3899874447612426)
('Total Profit %= ', 2.71621771288153)
('Total Profit %= ', 2.794230962742006)
('Total Profit %= ', 10.979585386408196)
('Total Profit %= ', 9.109209289116272)
('Total Profit %= ', 11.845926493835623)
('Total Profit %= ', 2.9222949183482783)
('Total Profit %= ', 11.09273387817591)
('Total Profit %= ', 10.757834286242485)
('Total Profit %= ', 11.229187960808563)
('Total Profit %= ', 2.559715557338693)
('Total Profit %= ', 9.177863223137116)
('Total Profit %= ', 

('Total Profit %= ', -3.377915330742276)
('Total Profit %= ', -4.780958951356739)
('Total Profit %= ', -1.8440991171466026)
('Total Profit %= ', -5.36221754512527)
('Total Profit %= ', -3.7722089663284386)
('Total Profit %= ', -4.904918092936716)
('Total Profit %= ', -0.9708424923587367)
('Total Profit %= ', -5.580524212208027)
('Total Profit %= ', -4.022896121581521)
('Total Profit %= ', -3.0546734156104316)
('Total Profit %= ', -2.1540764129073686)
('Total Profit %= ', -4.26662306503387)
('Total Profit %= ', -3.641293495865583)
('Total Profit %= ', -4.603657447201492)
('Total Profit %= ', -4.84590958909129)
('Total Profit %= ', -1.4423757882412347)
('Total Profit %= ', -2.8449894635141026)
('Total Profit %= ', -3.575993315836823)
('Total Profit %= ', -3.5821498234099525)
('Total Profit %= ', -1.521117806677319)
('Total Profit %= ', -1.6260646514116517)
('Total Profit %= ', -0.9031028521407469)
('Total Profit %= ', -2.091823089521072)
('Total Profit %= ', -1.4751014963040543)
('Total 

In [ ]:
import csv
f = csv.writer(open("result.csv", "wb+"))

f.writerow(["ST", "LT", "targetPercentage", "stopLossPercentage", "entryDifference","profitPercentage"])

for x in csvList:
    f.writerow([x['params'][2],
                x['params'][3],
                x['params'][4],
                x['params'][5],
                x['params'][6],
                x["result"]])

In [ ]:
import csv
f = csv.writer(open("result.csv", "wb+"))

f.writerow(["ST", "LT", "targetPercentage", "stopLossPercentage", "entryDifference","profitPercentage"])

for x in csvList:
    f.writerow([x['params'][2],
                x['params'][3],
                x['params'][4],
                x['params'][5],
                x['params'][6],
                x["result"]])